In [10]:
import xarray as xr
import numpy as np
import pandas as pd

In [11]:
ERA5 = xr.open_dataset('/home/m/m301036/josie/LSAT/1900-2022LSAT/SpatialPattern/SLP/slp_ERA5_ano.nc')
ERA5_ano =ERA5.sel(time=slice('1988-01-01','2022-12-31'))

ERA5_ano

<xarray.Dataset>
Dimensions:    (time: 420, longitude: 360, latitude: 91)
Coordinates:
  * time       (time) datetime64[ns] 1988-01-01 1988-02-01 ... 2022-12-01
  * longitude  (longitude) float32 0.0 1.0 2.0 3.0 ... 356.0 357.0 358.0 359.0
  * latitude   (latitude) float32 0.0 1.0 2.0 3.0 4.0 ... 87.0 88.0 89.0 90.0
    month      (time) int64 ...
Data variables:
    msl        (time, latitude, longitude) float64 ...

In [12]:
ERA5_ano['DJF'] = ERA5_ano['msl'].sel(time=ERA5_ano['time.season']=='DJF').groupby('time.year').mean('time')
ERA5_ano['MAM'] = ERA5_ano['msl'].sel(time=ERA5_ano['time.season']=='MAM').groupby('time.year').mean('time')
ERA5_ano['JJA'] = ERA5_ano['msl'].sel(time=ERA5_ano['time.season']=='JJA').groupby('time.year').mean('time')
ERA5_ano['SON'] = ERA5_ano['msl'].sel(time=ERA5_ano['time.season']=='SON').groupby('time.year').mean('time')

## Define function

In [13]:
from scipy.stats import linregress

def calc_trend(data):
    slope, intercept, r_value, p_value, std_err = linregress(np.arange(len(data)), data)  
    return slope, p_value

## ERA5 seasonal trend pattern

In [14]:
ERA5_ano['DJF_trend'], ERA5_ano['p_value_DJF'] = xr.apply_ufunc(calc_trend, ERA5_ano['DJF'], input_core_dims=[['year']], output_core_dims=[[], []], vectorize=True)
ERA5_ano['MAM_trend'], ERA5_ano['p_value_MAM'] = xr.apply_ufunc(calc_trend, ERA5_ano['MAM'], input_core_dims=[['year']], output_core_dims=[[], []], vectorize=True)
ERA5_ano['JJA_trend'], ERA5_ano['p_value_JJA'] = xr.apply_ufunc(calc_trend, ERA5_ano['JJA'], input_core_dims=[['year']], output_core_dims=[[], []], vectorize=True)
ERA5_ano['SON_trend'], ERA5_ano['p_value_SON'] = xr.apply_ufunc(calc_trend, ERA5_ano['SON'], input_core_dims=[['year']], output_core_dims=[[], []], vectorize=True)

In [15]:
SLP_trend = xr.Dataset({
    'DJF': ERA5_ano['DJF_trend']*35.0,
    'MAM': ERA5_ano['MAM_trend']*35.0,
    'JJA': ERA5_ano['JJA_trend']*35.0,
    'SON': ERA5_ano['SON_trend']*35.0
}, coords={'longitude': ERA5_ano['longitude'], 'latitude': ERA5_ano['latitude'], 'season': ['DJF', 'MAM', 'JJA', 'SON']})
SLP_trend

<xarray.Dataset>
Dimensions:    (longitude: 360, latitude: 91, season: 4)
Coordinates:
  * longitude  (longitude) float32 0.0 1.0 2.0 3.0 ... 356.0 357.0 358.0 359.0
  * latitude   (latitude) float32 0.0 1.0 2.0 3.0 4.0 ... 87.0 88.0 89.0 90.0
  * season     (season) <U3 'DJF' 'MAM' 'JJA' 'SON'
Data variables:
    DJF        (latitude, longitude) float64 -9.844 -10.68 -11.5 ... 447.5 447.5
    MAM        (latitude, longitude) float64 38.65 38.98 39.33 ... -14.09 -14.09
    JJA        (latitude, longitude) float64 -15.21 -16.09 ... 325.1 325.1
    SON        (latitude, longitude) float64 -24.42 -23.79 ... 103.6 103.6

In [16]:
sig_ds = xr.Dataset({
    'DJF': ERA5_ano['p_value_DJF'],
    'MAM': ERA5_ano['p_value_MAM'],
    'JJA': ERA5_ano['p_value_JJA'],
    'SON': ERA5_ano['p_value_SON']
}, coords={'longitude': ERA5_ano['longitude'], 'latitude': ERA5_ano['latitude'], 'season': ['DJF', 'MAM', 'JJA', 'SON']})
sig_ds

<xarray.Dataset>
Dimensions:    (longitude: 360, latitude: 91, season: 4)
Coordinates:
  * longitude  (longitude) float32 0.0 1.0 2.0 3.0 ... 356.0 357.0 358.0 359.0
  * latitude   (latitude) float32 0.0 1.0 2.0 3.0 4.0 ... 87.0 88.0 89.0 90.0
  * season     (season) <U3 'DJF' 'MAM' 'JJA' 'SON'
Data variables:
    DJF        (latitude, longitude) float64 0.7228 0.7 0.6782 ... 0.0636 0.0636
    MAM        (latitude, longitude) float64 0.1389 0.1338 ... 0.9614 0.9614
    JJA        (latitude, longitude) float64 0.5941 0.5728 ... 0.1665 0.1665
    SON        (latitude, longitude) float64 0.2406 0.2453 ... 0.5962 0.5962

## MPI-ESM-Trend Calculating

In [18]:
data_hist_ssp245_MPI_ESM  = '/work/mh0033/m301036/LSAT/CMIP6-MPI-M-LR/MergeDataOut/psl_Amon_1850-2022_*.nc'
ds = xr.open_mfdataset(data_hist_ssp245_MPI_ESM, combine = 'nested', concat_dim = 'run')
ds

<xarray.Dataset>
Dimensions:    (time: 2076, run: 30, bnds: 2, lon: 180, lat: 90)
Coordinates:
  * time       (time) datetime64[ns] 1850-01-16T12:00:00 ... 2022-12-16T12:00:00
  * lon        (lon) float64 0.0 2.0 4.0 6.0 8.0 ... 352.0 354.0 356.0 358.0
  * lat        (lat) float64 -89.0 -87.0 -85.0 -83.0 ... 83.0 85.0 87.0 89.0
Dimensions without coordinates: run, bnds
Data variables:
    time_bnds  (run, time, bnds) datetime64[ns] dask.array<chunksize=(1, 2076, 2), meta=np.ndarray>
    psl        (run, time, lat, lon) float32 dask.array<chunksize=(1, 2076, 90, 180), meta=np.ndarray>
Attributes: (12/49)
    CDI:                    Climate Data Interface version 2.0.3 (https://mpi...
    source:                 MPI-ESM1.2-LR (2017): \naerosol: none, prescribed...
    institution:            Max Planck Institute for Meteorology, Hamburg 201...
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            CMIP
    branch_method:          standard
    ...                     ...
    variable_id:            psl
    variant_label:          r1i1p1f1
    license:                CMIP6 model data produced by MPI-M is licensed un...
    cmor_version:           3.5.0
    tracking_id:            hdl:21.14100/2d4e90e1-d70a-49f4-8374-eb769a0b143d
    CDO:                    Climate Data Operators version 2.0.3 (https://mpi...

In [19]:
# #Perfrom land sea mask
# land_sea_mask=xr.open_dataset('/work/mh0033/m301036/LSAT/CMIP6-MPI-M-LR/GR15_lsm_regrid.nc')
# # land_sea_mask.coords
# display(land_sea_mask)
# # land_sea_mask.coords

In [20]:
# mask_data = land_sea_mask['var1']
# mask_data
# # Align the time coordinates between the mask dataset and the original dataset
# time_values = pd.to_datetime(mask_data['time'].values, format='mixed', dayfirst=True)
# mask_data['time'] = time_values

# # Align the time coordinates between the mask dataset and the original dataset
# mask_data = mask_data.reindex(time=ds['time'], method='nearest')

# # Apply the land-sea mask to the original dataset
# masked_tas = ds.where(mask_data == 1, drop=False)
# masked_tas

In [21]:
slp = ds['psl'].loc[:,'1988-01-01':'2022-12-31',0:90,:]
print(slp.min().values)
slp_climatology = slp.groupby('time.month').mean(dim='time')
slp_climatology
slp_ano = slp.groupby('time.month') - slp_climatology
slp_ano_ds = slp_ano.to_dataset()
slp_ano_ds

97214.24


/home/m/m301036/.conda/envs/mykernel/lib/python3.9/site-packages/xarray/core/indexing.py:1440: PerformanceWarning: Slicing with an out-of-order index is generating 35 times more chunks
  return self.array[key]


<xarray.Dataset>
Dimensions:  (time: 420, lon: 180, lat: 45, run: 30)
Coordinates:
  * time     (time) datetime64[ns] 1988-01-16T12:00:00 ... 2022-12-16T12:00:00
  * lon      (lon) float64 0.0 2.0 4.0 6.0 8.0 ... 350.0 352.0 354.0 356.0 358.0
  * lat      (lat) float64 1.0 3.0 5.0 7.0 9.0 11.0 ... 81.0 83.0 85.0 87.0 89.0
    month    (time) int64 1 2 3 4 5 6 7 8 9 10 11 ... 2 3 4 5 6 7 8 9 10 11 12
Dimensions without coordinates: run
Data variables:
    psl      (run, time, lat, lon) float32 dask.array<chunksize=(1, 1, 45, 180), meta=np.ndarray>

In [22]:
seasons = ['JJA', 'DJF', 'MAM', 'SON']
season_means = {}

for season in seasons:
    if season == 'JJA':
        months = [6,7,8]
    elif season == 'DJF':
        months =[12,1,2]
    elif season == 'MAM':
        months = [3,4,5]
    elif season == 'SON':
        months = [9,10,11]

    season_months = slp_ano_ds.sel(time=slp_ano_ds.time.dt.month.isin(months))
    
    # Calculate the seasonal mean SAT anomalies
    season_mean_anomalies = season_months.groupby('time.year').mean('time')
    
    # Store the seasonal mean in the dictionary
    season_means[season] = season_mean_anomalies['psl']

# Access the multiyear JJA mean SAT anomalies
    
slp_ano_ds['JJA'] = season_means['JJA']
slp_ano_ds['DJF'] = season_means['DJF']
slp_ano_ds['MAM'] = season_means['MAM']
slp_ano_ds['SON'] = season_means['SON']

slp_ano_ds

<xarray.Dataset>
Dimensions:  (time: 420, lon: 180, lat: 45, run: 30, year: 35)
Coordinates:
  * time     (time) datetime64[ns] 1988-01-16T12:00:00 ... 2022-12-16T12:00:00
  * lon      (lon) float64 0.0 2.0 4.0 6.0 8.0 ... 350.0 352.0 354.0 356.0 358.0
  * lat      (lat) float64 1.0 3.0 5.0 7.0 9.0 11.0 ... 81.0 83.0 85.0 87.0 89.0
    month    (time) int64 1 2 3 4 5 6 7 8 9 10 11 ... 2 3 4 5 6 7 8 9 10 11 12
  * year     (year) int64 1988 1989 1990 1991 1992 ... 2018 2019 2020 2021 2022
Dimensions without coordinates: run
Data variables:
    psl      (run, time, lat, lon) float32 dask.array<chunksize=(1, 1, 45, 180), meta=np.ndarray>
    JJA      (year, run, lat, lon) float32 dask.array<chunksize=(1, 1, 45, 180), meta=np.ndarray>
    DJF      (year, run, lat, lon) float32 dask.array<chunksize=(1, 1, 45, 180), meta=np.ndarray>
    MAM      (year, run, lat, lon) float32 dask.array<chunksize=(1, 1, 45, 180), meta=np.ndarray>
    SON      (year, run, lat, lon) float32 dask.array<chunksize=(1, 1, 45, 180), meta=np.ndarray>

In [23]:
import scipy.stats as stats
from scipy.stats import linregress

def calc_trend(data):
    slope, intercept, r_value, p_value, std_err = linregress(np.arange(len(data)), data)  
    return slope, p_value

In [24]:
slp_ano_ds['slope_JJA'], slp_ano_ds['p_value_JJA'] = xr.apply_ufunc(calc_trend, slp_ano_ds['JJA'].chunk(dict(run=-1)), input_core_dims=[['year']], output_core_dims=[[],[]], vectorize=True, dask='parallelized', output_dtypes=[float,float], dask_gufunc_kwargs={'allow_rechunk': True})
slp_ano_ds['slope_JJA'].attrs['units'] = 'Pa/35yrs'
slp_ano_ds['p_value_JJA'].attrs['units'] = 'p_value'

In [25]:
slp_ano_ds['slope_DJF'], slp_ano_ds['p_value_DJF'] = xr.apply_ufunc(calc_trend, slp_ano_ds['DJF'].chunk(dict(run=-1, year=-1)), input_core_dims=[['year']], output_core_dims=[[],[]], vectorize=True, dask='parallelized', output_dtypes=[float,float], dask_gufunc_kwargs={'allow_rechunk': True})
slp_ano_ds['slope_DJF'].attrs['units'] = 'Pa/35yrs'
slp_ano_ds['p_value_DJF'].attrs['units'] = 'p_value'

In [26]:
slp_ano_ds['slope_MAM'], slp_ano_ds['p_value_MAM'] = xr.apply_ufunc(calc_trend, slp_ano_ds['MAM'].chunk(dict(run=-1, year=-1)), input_core_dims=[['year']], output_core_dims=[[],[]], vectorize=True, dask='parallelized', output_dtypes=[float,float], dask_gufunc_kwargs={'allow_rechunk': True})
slp_ano_ds['slope_MAM'].attrs['units'] = 'Pa/35yrs'
slp_ano_ds['p_value_MAM'].attrs['units'] = 'p_value'

In [27]:
slp_ano_ds['slope_SON'], slp_ano_ds['p_value_SON'] = xr.apply_ufunc(calc_trend, slp_ano_ds['SON'].chunk(dict(run=-1, year=-1)), input_core_dims=[['year']], output_core_dims=[[],[]], vectorize=True, dask='parallelized', output_dtypes=[float,float], dask_gufunc_kwargs={'allow_rechunk': True})
slp_ano_ds['slope_SON'].attrs['units'] = 'Pa/35yrs'
slp_ano_ds['p_value_SON'].attrs['units'] = 'p_value'

In [28]:
slp_ano_ds = slp_ano_ds.compute()

In [29]:
slp_ano_ds

<xarray.Dataset>
Dimensions:      (time: 420, lon: 180, lat: 45, run: 30, year: 35)
Coordinates:
  * time         (time) datetime64[ns] 1988-01-16T12:00:00 ... 2022-12-16T12:...
  * lon          (lon) float64 0.0 2.0 4.0 6.0 8.0 ... 352.0 354.0 356.0 358.0
  * lat          (lat) float64 1.0 3.0 5.0 7.0 9.0 ... 81.0 83.0 85.0 87.0 89.0
    month        (time) int64 1 2 3 4 5 6 7 8 9 10 11 ... 3 4 5 6 7 8 9 10 11 12
  * year         (year) int64 1988 1989 1990 1991 1992 ... 2019 2020 2021 2022
Dimensions without coordinates: run
Data variables: (12/13)
    psl          (run, time, lat, lon) float32 8.922 8.922 ... -328.9 -330.2
    JJA          (year, run, lat, lon) float32 53.51 51.15 ... -282.7 -281.6
    DJF          (year, run, lat, lon) float32 9.865 10.04 8.232 ... 680.0 678.8
    MAM          (year, run, lat, lon) float32 13.44 15.62 ... -678.5 -680.5
    SON          (year, run, lat, lon) float32 3.727 2.154 2.854 ... 308.5 310.7
    slope_JJA    (run, lat, lon) float64 -0.09831 -0.09845 ... -4.785 -4.802
    ...           ...
    slope_DJF    (run, lat, lon) float64 -1.526 -1.534 -1.54 ... 6.381 6.365
    p_value_DJF  (run, lat, lon) float64 0.07907 0.07388 ... 0.3736 0.3751
    slope_MAM    (run, lat, lon) float64 -0.2286 -0.2283 -0.2087 ... 1.035 1.027
    p_value_MAM  (run, lat, lon) float64 0.7485 0.7484 0.771 ... 0.8564 0.8579
    slope_SON    (run, lat, lon) float64 -1.421 -1.384 -1.356 ... 4.484 4.531
    p_value_SON  (run, lat, lon) float64 0.04097 0.0441 ... 0.3686 0.3639

## Selection of the minimum and maximum five models

In [30]:
model_35yr_JJA = np.genfromtxt(fname='/home/m/m301036/josie/LSAT/1900-2022LSAT/MPI-ESM-LR_NH_SATAs_35yr_JJA_trend.txt',delimiter='\t', skip_header=1)
model_35yr_DJF = np.genfromtxt(fname='/home/m/m301036/josie/LSAT/1900-2022LSAT/MPI-ESM-LR_NH_SATAs_35yr_DJF_trend.txt',delimiter='\t', skip_header=1)

DJF_35yr = xr.DataArray(model_35yr_DJF[:,1], dims=['run'], coords={'run': np.arange(1, 31, 1)})
JJA_35yr = xr.DataArray(model_35yr_JJA[:,1], dims=['run'], coords={'run': np.arange(1, 31, 1)})

# assume you have an xarray called 'xr_data'
# get the most minimum five values and their indices
min_indices = np.argpartition(DJF_35yr.values.flatten(), 5)[:5]
min_values = DJF_35yr.values.flatten()[min_indices]
min_coords = np.unravel_index(min_indices, DJF_35yr.shape)

# create a new xarray to store the most minimum values
min_xr = xr.DataArray(min_values, dims=['value'], coords={'value': np.arange(5)})

# print the new xarray
print(min_xr)
print(min_coords)

<xarray.DataArray (value: 5)>
array([0.56793617, 0.66594894, 0.67352914, 0.68214545, 0.71430487])
Coordinates:
  * value    (value) int64 0 1 2 3 4
(array([16, 25, 19,  5, 20]),)


In [31]:
max_indices = np.argpartition(DJF_35yr.values.flatten(), -5)[-5:]
max_values = DJF_35yr.values.flatten()[max_indices]
max_coords = np.unravel_index(max_indices, DJF_35yr.shape)

max_xr = xr.DataArray(max_values, dims=['value'], coords={'value': np.arange(5)})
print(max_xr)
print(max_coords)
type(max_coords)
max_coords

<xarray.DataArray (value: 5)>
array([1.49677206, 1.50581782, 1.59931985, 1.7537178 , 2.05886683])
Coordinates:
  * value    (value) int64 0 1 2 3 4
(array([24, 27, 18,  1,  4]),)


(array([24, 27, 18,  1,  4]),)

In [32]:
max_JJA_indices = np.argpartition(JJA_35yr.values.flatten(), -5)[-5:]
max_JJA_values = JJA_35yr.values.flatten()[max_JJA_indices]
max_JJA_coords = np.unravel_index(max_JJA_indices, JJA_35yr.shape)

max_JJA_xr = xr.DataArray(max_JJA_values, dims=['value'], coords={'value': np.arange(5)})
print(max_JJA_xr)
print(max_JJA_coords)
type(max_JJA_coords)
max_JJA_coords

<xarray.DataArray (value: 5)>
array([1.21295413, 1.25939745, 1.24528245, 1.31655035, 1.35779019])
Coordinates:
  * value    (value) int64 0 1 2 3 4
(array([ 4, 16, 17, 15, 18]),)


(array([ 4, 16, 17, 15, 18]),)

In [33]:
min_JJA_indices = np.argpartition(JJA_35yr.values.flatten(), 5)[:5]
min_JJA_values = JJA_35yr.values.flatten()[min_JJA_indices]
min_JJA_coords = np.unravel_index(min_JJA_indices, JJA_35yr.shape)

min_JJA_xr = xr.DataArray(min_JJA_values, dims=['value'], coords={'value': np.arange(5)})
print(min_JJA_xr)
print(min_JJA_coords)
type(min_JJA_coords)
min_JJA_coords

<xarray.DataArray (value: 5)>
array([0.91451912, 0.88953555, 0.88910033, 0.87685294, 0.75293976])
Coordinates:
  * value    (value) int64 0 1 2 3 4
(array([ 0, 20,  9,  8, 25]),)


(array([ 0, 20,  9,  8, 25]),)

## Plot the trend pattern of the DJF 35-yr Spatial pattern

In [34]:
slope_JJA = slp_ano_ds['slope_JJA'] 
slope_DJF = slp_ano_ds['slope_DJF']
slope_MAM = slp_ano_ds['slope_MAM']
slope_SON = slp_ano_ds['slope_SON']

p_value_JJA = slp_ano_ds['p_value_JJA']
p_value_DJF = slp_ano_ds['p_value_DJF']
p_value_MAM = slp_ano_ds['p_value_MAM']
p_value_SON = slp_ano_ds['p_value_SON']

In [35]:
slope_JJA_data = slope_JJA*35.0
slope_DJF_data = slope_DJF*35.0
slope_MAM_data = slope_MAM*35.0
slope_SON_data = slope_SON*35.0

In [36]:
slope_DJF_data

<xarray.DataArray 'slope_DJF' (run: 30, lat: 45, lon: 180)>
array([[[ -53.42172191,  -53.68635133,  -53.89294864, ...,
          -53.17455595,  -54.14516895,  -54.93589151],
        [ -49.51092558,  -51.06009969,  -50.30384457, ...,
          -50.26248368,  -52.12418355,  -50.90847084],
        [ -40.85809885,  -43.50048472,  -41.46027365, ...,
          -42.21890414,  -45.42394261,  -42.17695491],
        ...,
        [ 550.08415548,  554.94400486,  559.44829584, ...,
          533.38411154,  539.27602401,  544.84378005],
        [ 580.05992248,  583.60580678,  586.96915082, ...,
          568.13030647,  572.2938057 ,  576.25832862],
        [ 571.00148026,  572.34936855,  573.64717038, ...,
          565.16565325,  566.68372898,  568.17271569]],

       [[  26.18195981,   26.22114993,   26.8770427 , ...,
           30.85153727,   29.53523274,   27.03311345],
        [  28.76616125,   25.82802253,   26.89100847, ...,
           31.14210557,   29.17519464,   29.71969463],
        [  39.20876695,   32.95618812,   33.19035822, ...,
           39.60534146,   36.77121685,   40.30055135],
...
        [-242.78500488, -247.1301102 , -251.09190998, ...,
         -227.76232946, -233.02440492, -238.05746258],
        [-214.70327951, -215.56717033, -216.31553474, ...,
         -211.33062607, -212.55167709, -213.68252066],
        [-184.45108905, -184.11978529, -183.74333929, ...,
         -185.3177563 , -185.17094475, -184.9676972 ]],

       [[  17.90992631,   18.86136667,   20.27195691, ...,
           21.77001465,   19.08067842,   17.06484893],
        [  25.00112422,   22.1280878 ,   25.35191863, ...,
           26.21377041,   25.25370275,   27.04248425],
        [  43.89187589,   35.69281072,   38.3048927 , ...,
           41.09260133,   42.44824856,   48.75653683],
        ...,
        [ 289.59071392,  286.4588067 ,  282.80764559, ...,
          296.12186825,  294.37203957,  292.20616941],
        [ 246.73457471,  244.7050166 ,  242.5993717 , ...,
          252.0795827 ,  250.41183139,  248.59492089],
        [ 221.63722949,  221.04008594,  220.41135398, ...,
          223.85693295,  223.32527533,  222.78628388]]])
Coordinates:
  * lon      (lon) float64 0.0 2.0 4.0 6.0 8.0 ... 350.0 352.0 354.0 356.0 358.0
  * lat      (lat) float64 1.0 3.0 5.0 7.0 9.0 11.0 ... 81.0 83.0 85.0 87.0 89.0
Dimensions without coordinates: run

In [37]:
slope_JJA_MME = slope_JJA_data.mean(dim='run')
slope_DJF_MME = slope_DJF_data.mean(dim='run')
slope_MAM_MME = slope_MAM_data.mean(dim='run')
slope_SON_MME = slope_SON_data.mean(dim='run')

In [38]:
#using the for loop to pick up the min5 and max5 data of DJF 


# Extract trend spatial data for minimum five runs in DJF
DJF_min5_trend = []
for i in range(5):
    run_index = min_coords[0][i]
    DJF_min5_trend.append(slope_DJF_data[run_index,:,:])
DJF_min5_trend = xr.concat(DJF_min5_trend, dim='run')

# Extract trend spatial data for maximum five runs in DJF
DJF_max5_trend = []
for i in range(5):
    run_index = max_coords[0][i]
    DJF_max5_trend.append(slope_DJF_data[run_index,:,:])
DJF_max5_trend = xr.concat(DJF_max5_trend, dim='run')

# Extract trend spatial data for minimum five runs in JJA
JJA_min5_trend = []
for i in range(5):
    run_index = min_JJA_coords[0][i]
    JJA_min5_trend.append(slope_JJA_data[run_index,:,:])
JJA_min5_trend = xr.concat(JJA_min5_trend, dim='run')

# Extract trend spatial data for maximum five runs in JJA
JJA_max5_trend = []
for i in range(5):
    run_index = max_JJA_coords[0][i]
    JJA_max5_trend.append(slope_JJA_data[run_index,:,:])
JJA_max5_trend = xr.concat(JJA_max5_trend, dim='run')


In [39]:
JJA_max5_trend

<xarray.DataArray 'slope_JJA' (run: 5, lat: 45, lon: 180)>
array([[[-1.41423610e+01, -1.74578233e+01, -1.80702099e+01, ...,
         -1.07270478e+01, -1.34126839e+01, -1.33892203e+01],
        [-1.39458746e+01, -1.75100337e+01, -1.77816074e+01, ...,
         -1.08434939e+01, -1.20040082e+01, -1.17269453e+01],
        [-1.93979777e+01, -2.06741209e+01, -1.97357527e+01, ...,
         -1.13235289e+01, -1.18971358e+01, -1.50609174e+01],
        ...,
        [ 3.00957710e+00,  8.20043648e-02, -2.80246834e+00, ...,
          1.20247398e+01,  8.99711679e+00,  5.98238274e+00],
        [ 2.72750222e+01,  2.61337895e+01,  2.50027564e+01, ...,
          3.08844956e+01,  2.96632490e+01,  2.84649994e+01],
        [ 4.66484380e+01,  4.63840148e+01,  4.61390186e+01, ...,
          4.78298351e+01,  4.75025028e+01,  4.72040919e+01]],

       [[-1.59162074e+01, -1.78063224e+01, -1.54721202e+01, ...,
         -1.93403539e+01, -1.89118926e+01, -1.62534215e+01],
        [-1.79733960e+01, -1.87196943e+01, -1.77103760e+01, ...,
         -2.05709762e+01, -2.10023226e+01, -1.90664824e+01],
        [-1.78922071e+01, -1.60541768e+01, -1.56304891e+01, ...,
         -2.00398803e+01, -2.00311481e+01, -1.96315366e+01],
...
        [ 6.09886088e+01,  6.51620217e+01,  6.89369713e+01, ...,
          4.57583238e+01,  5.12192070e+01,  5.62926072e+01],
        [ 7.79043958e+00,  9.55190900e+00,  1.11041236e+01, ...,
          1.48190045e+00,  3.77641983e+00,  5.90139993e+00],
        [-5.02729778e+01, -4.97187952e+01, -4.92220495e+01, ...,
         -5.31457831e+01, -5.23281004e+01, -5.15794250e+01]],

       [[ 7.50702162e+00,  6.41084686e+00,  6.40660718e+00, ...,
          7.55362030e-01,  9.20496017e-01,  3.63993476e+00],
        [ 8.71645236e+00,  8.95646960e+00,  7.13117808e+00, ...,
         -1.22122746e+00, -2.25286670e-01,  3.21675903e+00],
        [ 1.30645679e+01,  1.34241211e+01,  1.08181429e+01, ...,
          6.80823612e+00,  9.07250800e+00,  1.03126528e+01],
        ...,
        [-8.70940056e+01, -8.58643796e+01, -8.47019273e+01, ...,
         -9.10009676e+01, -8.96485386e+01, -8.83308315e+01],
        [-8.17723144e+01, -8.07716529e+01, -7.98488306e+01, ...,
         -8.46397345e+01, -8.36844063e+01, -8.27338379e+01],
        [-7.35337773e+01, -7.31410785e+01, -7.27366458e+01, ...,
         -7.50388094e+01, -7.46792044e+01, -7.43042039e+01]]])
Coordinates:
  * lon      (lon) float64 0.0 2.0 4.0 6.0 8.0 ... 350.0 352.0 354.0 356.0 358.0
  * lat      (lat) float64 1.0 3.0 5.0 7.0 9.0 11.0 ... 81.0 83.0 85.0 87.0 89.0
Dimensions without coordinates: run

In [40]:
# Calculate the DJF min 5 trend mean
DJF_min5_trend_mean = DJF_min5_trend.mean(dim='run')
DJF_max5_trend_mean = DJF_max5_trend.mean(dim='run')

# Calculate the JJA min 5 trend mean
JJA_min5_trend_mean = JJA_min5_trend.mean(dim='run')
JJA_max5_trend_mean = JJA_max5_trend.mean(dim='run')

In [46]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.ticker as mticker
import cartopy.feature as cfeature
import cartopy.mpl.ticker as cticker
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
import matplotlib.gridspec as gridspec
import matplotlib as mpl
import seaborn as sns
from matplotlib.colors import ListedColormap
from matplotlib.colors import BoundaryNorm, ListedColormap

# plt.style.use('ggplot')
# levels = np.arange(-2.0, 5.5, 0.5)
# sns.color_palette("rocket_r", as_cmap=True)
# cmap = plt.cm.get_cmap('rocket_r', len(levels) - 1)
# bounds = [-2, -1, 0, 1, 2, 3, 4, 5]
# norm = mpl.colors.BoundaryNorm(bounds, cmap.N, extend='both')
levels = np.arange(-1000.0, 1050.0, 100.0)
cmap = ListedColormap(sns.color_palette("RdBu_r", n_colors=len(levels)-1).as_hex())

fig = plt.figure(figsize=(15, 8), dpi=300)
gs = gridspec.GridSpec(nrows=2, ncols=2, height_ratios=[1, 1], width_ratios=[1, 1])

ax1 = fig.add_subplot(gs[0, 0], projection=ccrs.PlateCarree(central_longitude=180))
ax1.set_extent([-180, 180, 0, 90], crs=ccrs.PlateCarree())
ax1.coastlines()
ax1.add_feature(cfeature.LAND, facecolor='lightgrey')
ax1.add_feature(cfeature.OCEAN, facecolor='white')

ax1.set_title('a', loc='left', fontsize=12, fontweight='medium')
ax1.set_title('MLOST DJF', loc='right', fontsize=12, fontweight='medium')

ax1.set_xticks([-180, -120, -60, 0, 60, 120, 180], crs=ccrs.PlateCarree())
ax1.set_yticks([0, 30, 60, 90], crs=ccrs.PlateCarree())
ax1.xaxis.set_major_formatter(cticker.LongitudeFormatter())
ax1.yaxis.set_major_formatter(cticker.LatitudeFormatter())

ax1.tick_params(axis='both', which='major', labelsize=10, direction='out', pad=5)

cf = ax1.contourf(SLP_trend['longitude'], SLP_trend['latitude'], SLP_trend['DJF'], levels=levels, cmap=cmap, transform=ccrs.PlateCarree())
# cb = plt.colorbar(cf, ax=ax1, orientation='horizontal', pad=0.05, shrink=0.8, extend='both')
# cb.set_label('Pa/35yrs', fontsize=10, fontweight='medium')
# cb.ax.tick_params(labelsize=10)

ax2 = fig.add_subplot(gs[0, 1], projection=ccrs.PlateCarree(central_longitude=180))
ax2.set_extent([-180, 180, 0, 90], crs=ccrs.PlateCarree())
ax2.coastlines()
ax2.add_feature(cfeature.LAND, facecolor='lightgrey')
ax2.add_feature(cfeature.OCEAN, facecolor='white')

ax2.set_title('b', loc='left', fontsize=12, fontweight='medium')
ax2.set_title('MPI-ESM-LR DJF MME', loc='right', fontsize=12, fontweight='medium')

ax2.set_xticks([-180, -120, -60, 0, 60, 120, 180], crs=ccrs.PlateCarree())
ax2.set_yticks([0, 30, 60, 90], crs=ccrs.PlateCarree())
ax2.xaxis.set_major_formatter(cticker.LongitudeFormatter())
ax2.yaxis.set_major_formatter(cticker.LatitudeFormatter())

ax2.tick_params(axis='both', which='major', labelsize=10, direction='out', pad=5)

cf = ax2.contourf(slope_DJF_data['lon'], slope_DJF_data['lat'], slope_DJF_MME, levels=levels, cmap=cmap, transform=ccrs.PlateCarree())
# cb = plt.colorbar(cf, ax=ax2, orientation='horizontal', pad=0.05, shrink=0.8, extend='both')
# cb.set_label('°C/65yrs', fontsize=10, fontweight='medium')
# cb.ax.tick_params(labelsize=10)

ax3 = fig.add_subplot(gs[1, 0], projection=ccrs.PlateCarree(central_longitude=180))
ax3.set_extent([-180, 180, 0, 90], crs=ccrs.PlateCarree())
ax3.coastlines()
ax3.add_feature(cfeature.LAND, facecolor='lightgrey')
ax3.add_feature(cfeature.OCEAN, facecolor='white')

ax3.set_title('c', loc='left', fontsize=12, fontweight='medium')
ax3.set_title('MPI-ESM-LR DJF Min5-MME', loc='right', fontsize=12, fontweight='medium')

ax3.set_xticks([-180, -120, -60, 0, 60, 120, 180], crs=ccrs.PlateCarree())
ax3.set_yticks([0, 30, 60, 90], crs=ccrs.PlateCarree())
ax3.xaxis.set_major_formatter(cticker.LongitudeFormatter())
ax3.yaxis.set_major_formatter(cticker.LatitudeFormatter())

ax3.tick_params(axis='both', which='major', labelsize=10, direction='out', pad=5)

cf = ax3.contourf(slope_DJF_data['lon'], slope_DJF_data['lat'], DJF_min5_trend_mean , levels=levels, cmap=cmap, transform=ccrs.PlateCarree())

ax4 = fig.add_subplot(gs[1, 1], projection=ccrs.PlateCarree(central_longitude=180))
ax4.set_extent([-180, 180, 0, 90], crs=ccrs.PlateCarree())
ax4.coastlines()
ax4.add_feature(cfeature.LAND, facecolor='lightgrey')
ax4.add_feature(cfeature.OCEAN, facecolor='white')

ax4.set_title('d', loc='left', fontsize=12, fontweight='medium')
ax4.set_title('MPI-ESM-LR DJF Max5-MME', loc='right', fontsize=12, fontweight='medium')

ax4.set_xticks([-180, -120, -60, 0, 60, 120, 180], crs=ccrs.PlateCarree())
ax4.set_yticks([0, 30, 60, 90], crs=ccrs.PlateCarree())
ax4.xaxis.set_major_formatter(cticker.LongitudeFormatter())
ax4.yaxis.set_major_formatter(cticker.LatitudeFormatter())

ax4.tick_params(axis='both', which='major', labelsize=10, direction='out', pad=5)

cf = ax4.contourf(slope_DJF_data['lon'], slope_DJF_data['lat'], DJF_max5_trend_mean, levels=levels, cmap=cmap, transform=ccrs.PlateCarree())

plt.subplots_adjust(wspace=0.1, hspace=0.01, top=0.75, bottom=0.05)

cb = plt.colorbar(cf, ax=[ax1, ax2, ax3, ax4], orientation='horizontal', pad=0.05, shrink=0.5, extend='both')
cb.set_label('Pa/35yrs', fontsize=10, fontweight='medium')
cb.ax.tick_params(labelsize=10)

plt.savefig('Obs-MPI-ESM-LR_1988-2022_DJF_SLP_trend_MME.png', bbox_inches='tight', dpi=300)

plt.close()

/tmp/ipykernel_1840613/2158247343.py:107: MatplotlibDeprecationWarning: The 'extend' parameter to Colorbar has no effect because it is overridden by the mappable; it is deprecated since 3.3 and will be removed two minor releases later.
  cb = plt.colorbar(cf, ax=[ax1, ax2, ax3, ax4], orientation='horizontal', pad=0.05, shrink=0.5, extend='both')


In [ ]:
# # Define the levels and colormap
levels = np.arange(-500.0, 550.0, 50.0)
cmap = ListedColormap(sns.color_palette("RdBu_r", n_colors=len(levels)-1).as_hex())

fig = plt.figure(figsize=(15, 8), dpi=300)
gs = gridspec.GridSpec(nrows=2, ncols=2, height_ratios=[1, 1], width_ratios=[1, 1])

ax1 = fig.add_subplot(gs[0, 0], projection=ccrs.PlateCarree(central_longitude=180))
ax1.set_extent([-180, 180, 0, 90], crs=ccrs.PlateCarree())
ax1.coastlines()
ax1.add_feature(cfeature.LAND, facecolor='lightgrey')
ax1.add_feature(cfeature.OCEAN, facecolor='white')

ax1.set_title('a', loc='left', fontsize=12, fontweight='medium')
ax1.set_title('MLOST JJA', loc='right', fontsize=12, fontweight='medium')

ax1.set_xticks([-180, -120, -60, 0, 60, 120, 180], crs=ccrs.PlateCarree())
ax1.set_yticks([0, 30, 60, 90], crs=ccrs.PlateCarree())
ax1.xaxis.set_major_formatter(cticker.LongitudeFormatter())
ax1.yaxis.set_major_formatter(cticker.LatitudeFormatter())

ax1.tick_params(axis='both', which='major', labelsize=10, direction='out', pad=5)

cf = ax1.contourf(SLP_trend['longitude'], SLP_trend['latitude'], SLP_trend['JJA'], levels=levels, cmap=cmap, transform=ccrs.PlateCarree())
# cb = plt.colorbar(cf, ax=ax1, orientation='horizontal', pad=0.05, shrink=0.8, extend='both')
# cb.set_label('°C/65yrs', fontsize=10, fontweight='medium')
# cb.ax.tick_params(labelsize=10)

ax2 = fig.add_subplot(gs[0, 1], projection=ccrs.PlateCarree(central_longitude=180))
ax2.set_extent([-180, 180, 0, 90], crs=ccrs.PlateCarree())
ax2.coastlines()
ax2.add_feature(cfeature.LAND, facecolor='lightgrey')
ax2.add_feature(cfeature.OCEAN, facecolor='white')

ax2.set_title('b', loc='left', fontsize=12, fontweight='medium')
ax2.set_title('MPI-ESM-LR JJA MME', loc='right', fontsize=12, fontweight='medium')

ax2.set_xticks([-180, -120, -60, 0, 60, 120, 180], crs=ccrs.PlateCarree())
ax2.set_yticks([0, 30, 60, 90], crs=ccrs.PlateCarree())
ax2.xaxis.set_major_formatter(cticker.LongitudeFormatter())
ax2.yaxis.set_major_formatter(cticker.LatitudeFormatter())

ax2.tick_params(axis='both', which='major', labelsize=10, direction='out', pad=5)

cf = ax2.contourf(slope_JJA_data['lon'], slope_JJA_data['lat'], slope_JJA_MME, levels=levels, cmap=cmap, transform=ccrs.PlateCarree())
# cb = plt.colorbar(cf, ax=ax2, orientation='horizontal', pad=0.05, shrink=0.8, extend='both')
# cb.set_label('°C/65yrs', fontsize=10, fontweight='medium')
# cb.ax.tick_params(labelsize=10)

ax3 = fig.add_subplot(gs[1, 0], projection=ccrs.PlateCarree(central_longitude=180))
ax3.set_extent([-180, 180, 0, 90], crs=ccrs.PlateCarree())
ax3.coastlines()
ax3.add_feature(cfeature.LAND, facecolor='lightgrey')
ax3.add_feature(cfeature.OCEAN, facecolor='white')

ax3.set_title('c', loc='left', fontsize=12, fontweight='medium')
ax3.set_title('MPI-ESM-LR JJA Min5-MME', loc='right', fontsize=12, fontweight='medium')

ax3.set_xticks([-180, -120, -60, 0, 60, 120, 180], crs=ccrs.PlateCarree())
ax3.set_yticks([0, 30, 60, 90], crs=ccrs.PlateCarree())
ax3.xaxis.set_major_formatter(cticker.LongitudeFormatter())
ax3.yaxis.set_major_formatter(cticker.LatitudeFormatter())

ax3.tick_params(axis='both', which='major', labelsize=10, direction='out', pad=5)

cf = ax3.contourf(slope_JJA_data['lon'], slope_JJA_data['lat'], JJA_min5_trend_mean , levels=levels, cmap=cmap, transform=ccrs.PlateCarree())

ax4 = fig.add_subplot(gs[1, 1], projection=ccrs.PlateCarree(central_longitude=180))
ax4.set_extent([-180, 180, 0, 90], crs=ccrs.PlateCarree())
ax4.coastlines()
ax4.add_feature(cfeature.LAND, facecolor='lightgrey')
ax4.add_feature(cfeature.OCEAN, facecolor='white')

ax4.set_title('d', loc='left', fontsize=12, fontweight='medium')
ax4.set_title('MPI-ESM-LR JJA Max5-MME', loc='right', fontsize=12, fontweight='medium')

ax4.set_xticks([-180, -120, -60, 0, 60, 120, 180], crs=ccrs.PlateCarree())
ax4.set_yticks([0, 30, 60, 90], crs=ccrs.PlateCarree())
ax4.xaxis.set_major_formatter(cticker.LongitudeFormatter())
ax4.yaxis.set_major_formatter(cticker.LatitudeFormatter())

ax4.tick_params(axis='both', which='major', labelsize=10, direction='out', pad=5)

cf = ax4.contourf(slope_JJA_data['lon'], slope_JJA_data['lat'], JJA_max5_trend_mean, levels=levels, cmap=cmap, transform=ccrs.PlateCarree())

plt.subplots_adjust(wspace=0.1, hspace=0.01, top=0.75, bottom=0.05)

cb = plt.colorbar(cf, ax=[ax1, ax2, ax3, ax4], orientation='horizontal', pad=0.05, shrink=0.5, extend='both')
cb.set_label('Pa/35yrs', fontsize=10, fontweight='medium')
cb.ax.tick_params(labelsize=10)

plt.savefig('Obs-MPI-ESM-LR_1988-2022_JJA_SLP_trend_MME.png', bbox_inches='tight', dpi=300)

plt.close()